# Potential Field Environment

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()
from IPython.display import HTML

In [2]:
import mantrap
import numpy as np
import torch

env = mantrap.environment.PotentialFieldEnvironment(
    ego_type=mantrap.agents.IntegratorDTAgent, 
    ego_kwargs={"position": torch.zeros(2), "velocity": torch.zeros(2)}, 
    dt=0.4
)
env.add_ado(position=torch.tensor([-5, 2.0]), velocity=torch.tensor([1, 0]))

# Compute ego trajectories to base comparison on.
t_horizon = 9
ego_controls = [
    torch.zeros((t_horizon, 2), requires_grad=True),
    torch.tensor([[0.0, -1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[1.0, -1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[-1.0, 0.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[0.0, 1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
]
colors = ["red", "blue", "green", "yellow", "orange"]
assert len(ego_controls) == len(colors)

n = len(ego_controls)
ego_trajectories = [env.ego.unroll_trajectory(controls, dt=env.dt) for controls in ego_controls]

# Compute trajectory predictions in response to different ego actions.
ado_trajectory_wo = env.predict_wo_ego(t_horizon=t_horizon + 1).squeeze()
ado_trajectories_w = [env.predict_w_trajectory(trajectory).squeeze() for trajectory in ego_trajectories]

In [3]:
from mantrap.visualization import visualize_trajectories

labels_all = ["ado_wo", *[f"ado_{i + 1}" for i in range(n)], *[f"ego_{i + 1}" for i in range(n)]]
colors_all = ["black", *colors, *colors]

HTML(visualize_trajectories([ado_trajectory_wo, *ado_trajectories_w, *ego_trajectories],
                             labels=labels_all, colors=colors_all, env_axes=env.axes))

In [4]:
# Determine position-interactive objective and gradient for both scenarios.
cost = mantrap.modules.InteractionPositionModule(env=env, t_horizon=10)

print("Objectives")
for trajectory, color in zip(ego_trajectories, colors):
    objective = cost.objective(trajectory)
    print(f"{color} ==> {objective}")
    
print("\nGradients (L2-norm)")
for controls, trajectory, color in zip(ego_controls, ego_trajectories, colors):
    gradient = np.linalg.norm(cost.gradient(trajectory, grad_wrt=controls))
    print(f"{color} ==> {gradient}")

Objectives
red ==> 13.02173900604248
blue ==> 8.986250877380371
green ==> 8.775104522705078
yellow ==> 14.882181167602539
orange ==> 22.93802833557129

Gradients (L2-norm)
red ==> 3.593228340148926
blue ==> 1.7244787216186523
green ==> 1.7474673986434937
yellow ==> 3.89284086227417
orange ==> 7.874989032745361


Comment: A planning horizon `t_planning <= 2` will not affect the interaction objective, given some environment which models the ados as double integrators, such as the `PotentialFieldEnvironment` used in this example. This is due to the fact, that the position of a double integrator merely is affected by some force acting on it one  time-step later (see double integrator dynamics).

In [5]:
solver = mantrap.solver.SGradSolver(env, goal=torch.zeros(2), t_planning=t_horizon,
                                    modules=[(mantrap.modules.InteractionPositionModule, None)])

HTML(solver.visualize_heat_map(propagation="constant"))

In [6]:
solver = mantrap.solver.SGradSolver(env, goal=torch.zeros(2), t_planning=t_horizon,
                                    modules=[(mantrap.modules.InteractionAccelerationModule, None)])

HTML(solver.visualize_heat_map(propagation="constant"))